In [1]:
%load_ext autoreload
%autoreload 2

import polars as pl
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches
plt.style.use('ggplot')

import numpy as np
import utils


url_data = "./data"

In [2]:
%reload_ext autoreload

In [2]:
os.chdir('../')

In [3]:
speed_high = 15
speed_low = 2
speed_moderate_down = 6
speed_moderate_up = 8
utils.set_speed(speed_high,speed_low,speed_moderate_down,speed_moderate_up)

print(utils.speed_high)
print(utils.speed_low)
print(utils.speed_moderate_up)
print(utils.speed_moderate_down)

utils.get_speeds()

15
2
8
6
15
2
8
6


# Count number of events

In [ ]:
range_cut = 96
path_save = os.path.join(url_data,"dataset_split_events")

years = ["2000","2001","2002","2003","2004","2005","2006","2007","2008","2009",\
         "2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020"]

for year in years:
    print(year)
    try:
        print(utils.count_event_per_file(os.path.join(url_data,"dataset_split_events",f"{year}_{range_cut}_15_high.csv"))['events'].sum())
    except:
        print("empty csv")
    try:
        print(utils.count_event_per_file(os.path.join(url_data,"dataset_split_events",f"{year}_{range_cut}_moderate.csv"))['events'].sum())
    except:
        print("empty csv")
    try:
        print(utils.count_event_per_file(os.path.join(url_data,"dataset_split_events",f"{year}_{range_cut}_low.csv"))['events'].sum())
    except:
        print("empty csv")

# Reduce low and Moderate to the amount of high

Maybe I have to change how i select the events and i did random
- Random
- Same distribution as high

In [4]:
import random

range_cut = 96

years = ["2000","2006","2009","2010","2013","2014","2017"]
kinds = ["moderate","low"]
path_read = os.path.join(url_data,"dataset_split_events")

amount = [2385,3154,2204,3876,6387,3542,6037]

for index,year in enumerate(years):
    for kind in kinds:
        n_to_take = int(amount[index]/38)
        df = pl.read_csv(os.path.join(path_read,f"{year}_{range_cut}_{kind}.csv"))
        new_df = pl.DataFrame()
        for windfarm in range(0,38):
            # print("windfarm nº",windfarm)
            df_new = df.filter(pl.col("index") == windfarm)
            n_events = df_new["n_event"].unique().shape[0]
            random_indices = random.sample(range(1,n_events+1), n_to_take)
            df_selected = df_new.filter(df_new['n_event'].is_in(random_indices))
            new_df = pl.concat([new_df,df_selected], rechunk=True)
        new_df.write_csv(os.path.join(url_data,"cut_dataset",f"{year}_{range_cut}_{kind}_cut.csv"))


# Join cut in 3 files

In [6]:
path = os.path.join(url_data,"cut_dataset")

kinds = ["high","moderate","low"]
years = ["2000","2006","2009","2010","2013","2014","2017"]

range_cut = 96

for kind in kinds:
    print(kind)
    new_df = pl.DataFrame()
    for index,year in enumerate(years):
        print(year)
        if kind == "high":
            df = pl.read_csv(os.path.join(path,f"{year}_{range_cut}_15_{kind}.csv"))
        else:
            df = pl.read_csv(os.path.join(path,f"{year}_{range_cut}_{kind}_cut.csv"))
        df = df.with_column(pl.lit(year).alias("year"))
        # print(df.columns)
        new_df = pl.concat([new_df,df], rechunk=True)
        # print(new_df.columns)

    new_df = new_df.select(['time', 'index', 'n_event', 'year', 'cc', 'o3', 'pv', 'cape', 'blh', 'd2m', 'z', 'relative_humidity', 't2m', 't100m', 't135m', 'wdir100m', 'wspeed135m', 'wspeed100m'])
    new_df.write_csv(os.path.join(url_data,f"final_files",str(range_cut),f"{kind}.csv"))

high
2000
['level', 'time', 'index', 'cc', 'o3', 'pv', 'cape', 'blh', 'd2m', 'z', 'relative_humidity', 't2m', 't100m', 't135m', 'wdir100m', 'wspeed135m', 'wspeed100m', 'n_event', 'year']
['level', 'time', 'index', 'cc', 'o3', 'pv', 'cape', 'blh', 'd2m', 'z', 'relative_humidity', 't2m', 't100m', 't135m', 'wdir100m', 'wspeed135m', 'wspeed100m', 'n_event', 'year']
2006
['level', 'time', 'index', 'cc', 'o3', 'pv', 'cape', 'blh', 'd2m', 'z', 'relative_humidity', 't2m', 't100m', 't135m', 'wdir100m', 'wspeed135m', 'wspeed100m', 'n_event', 'year']
['level', 'time', 'index', 'cc', 'o3', 'pv', 'cape', 'blh', 'd2m', 'z', 'relative_humidity', 't2m', 't100m', 't135m', 'wdir100m', 'wspeed135m', 'wspeed100m', 'n_event', 'year']
2009
['level', 'time', 'index', 'cc', 'o3', 'pv', 'cape', 'blh', 'd2m', 'z', 'relative_humidity', 't2m', 't100m', 't135m', 'wdir100m', 'wspeed135m', 'wspeed100m', 'n_event', 'year']
['level', 'time', 'index', 'cc', 'o3', 'pv', 'cape', 'blh', 'd2m', 'z', 'relative_humidity', 't